# Data Cleaning

### Import Relevant Libararies

In [ ]:
import pandas as pd
import numpy as np
import nfl_data_py as nfl

In [45]:
# Read in web scrapped data
df = pd.read_csv('./Data/scrapeddraft0522.csv',index_col=0)

In [46]:
# Remove players who don't have a player name (they did not have any statistics)
df = df[df.player != '<th aria-label="Player" class="sort_default_asc show_partial_when_sorting left" data-stat="player">Player</th>']

### Clean up player names for the players that some of the web scraping code couldn't extract properly

In [47]:
df.player[df.player.str.contains('=')] = df.player[df.player.str.contains('=')].apply(lambda x: str(x).split(">")[1].split("<")[0])

### Split team round and pick text into separate columns

In [48]:
df[['team','round','pick']] = df['draft_info'].apply(lambda x: pd.Series(str(x).split('/')[0:3]))

In [49]:
df['round'] = df['round'].apply(lambda x: pd.Series(str(x).strip()))

In [50]:
# Remove rows that do not have any data
df = df[df['round'] != '']

### Clean the Round column and set undrafted players to round 8

In [51]:
df['round'] = df['round'].apply(lambda x: pd.Series(str(x)[0]))

In [52]:
df['round'][df['round']=='n'] = 8

C:\Users\capta\AppData\Local\Temp\ipykernel_21532\1221531417.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['round'][df['round']=='n'] = 8


### Clean the pick column to remove the text and just get the number

In [53]:
df['pick'] = df['pick'].apply(lambda x: pd.Series(str(x).strip().split(' ')[0]))

In [54]:
df['pick'] = df['pick'].apply(lambda x: pd.Series(str(x)[0:-2]))

In [55]:
df['pick'][df['pick'] == ''] = np.nan

C:\Users\capta\AppData\Local\Temp\ipykernel_21532\1104288695.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pick'][df['pick'] == ''] = np.nan


### Convert the feet-inches string into numeric inches

In [56]:
df[['feet','inches']] = df['height'].apply(lambda x: pd.Series(str(x).split('-')))

In [57]:
df['feet'] = pd.to_numeric(df.feet,errors='coerce')
df['inches'] = pd.to_numeric(df.inches,errors='coerce')
df['height'] = (df.feet * 12) + df.inches

In [58]:
df.drop(['inches','feet'],axis=1,inplace=True)

### Remove text columns that are no longer needed

In [59]:
df.drop(['link','draft_info'],axis = 1, inplace=True)

### Add NFL Python Dataset

In [60]:
# Call NFL Players Python pacakge
player = nfl.import_players()

# Find all players who were drafted in 2005 or later
draft_players = player.loc[(player.entry_year > 2004) & (player.draft_number >= 1)].sort_values(by=['entry_year','draft_number'])

# Select relevant columns of draft players
draft_player = draft_players[['display_name','position_group','entry_year','college_conference','draft_number']]

### Convert features into proper data types

In [61]:
cat_cols = ['player','pos','school_name','team']

In [62]:
# Find all numerical columns
num_cols = [col for col in df if col not in cat_cols]

In [63]:
# Convert numerical columns to float
df[num_cols] = df[num_cols].apply(pd.to_numeric,errors='coerce', axis=1)

### Add age column

I only had age for drafted players so I added the age to the drafted players

In [64]:
drafted = df[df['round'] != 8]

In [65]:
draft = df.merge(draft_player,how='left',left_on=['year','pick'],right_on=['entry_year','draft_number'])

In [66]:
age = pd.read_excel('./Data/Age.xlsx')

In [67]:
age = age[['Year','Rnd','Pick','Age','Player']]

In [68]:
age[['Year','Rnd','Pick','Age']] = age[['Year','Rnd','Pick','Age']].apply(pd.to_numeric,errors='coerce',axis=1)

In [69]:
drafts = draft.merge(age,how='left',left_on=['year','pick','player'],right_on=['Year','Pick','Player'])

### Remove unnecessary columns from merged DataFrame

In [70]:
drafts.drop(['display_name','position_group','entry_year','draft_number','Year','Rnd','Pick','Player'],axis=1,inplace=True)

### Cleaned up Position Labeling

In [71]:
drafts.pos[(drafts.pos == 'OLB')|(drafts.pos == 'ILB')] = 'LB'
drafts.pos[(drafts.pos == 'OT')|(drafts.pos == 'C')|(drafts.pos == 'OG')] = 'OL'

C:\Users\capta\AppData\Local\Temp\ipykernel_21532\1009202155.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.pos[(drafts.pos == 'OLB')|(drafts.pos == 'ILB')] = 'LB'
C:\Users\capta\AppData\Local\Temp\ipykernel_21532\1009202155.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.pos[(drafts.pos == 'OT')|(drafts.pos == 'C')|(drafts.pos == 'OG')] = 'OL'


In [72]:
# Add column to update round to Round for less code confusion
drafts['Round'] = drafts['round']

In [73]:
# Remove old round column
drafts.drop(['round'],axis=1,inplace=True)

### Cleaning up College Conferences

In [74]:
drafts.college_conference[(drafts.college_conference == 'NO COLLEGE') & (drafts.school_name == 'Ohio State')] = 'Big Ten Conference'

C:\Users\capta\AppData\Local\Temp\ipykernel_21532\1151124999.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.college_conference[(drafts.college_conference == 'NO COLLEGE') & (drafts.school_name == 'Ohio State')] = 'Big Ten Conference'


In [75]:
drafts.college_conference[(drafts.college_conference == 'NO COLLEGE')] = 'Atlantic Coast Conference'

C:\Users\capta\AppData\Local\Temp\ipykernel_21532\3583540359.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.college_conference[(drafts.college_conference == 'NO COLLEGE')] = 'Atlantic Coast Conference'


In [76]:
drafts.college_conference[(drafts.college_conference == 'NO FOOTBALL')] = 'Conference USA'

C:\Users\capta\AppData\Local\Temp\ipykernel_21532\2083932286.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.college_conference[(drafts.college_conference == 'NO FOOTBALL')] = 'Conference USA'


In [77]:
drafts.college_conference[(drafts.college_conference == 'Pacific Ten Conference')] = 'Pacific Twelve Conference'

C:\Users\capta\AppData\Local\Temp\ipykernel_21532\3775298207.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.college_conference[(drafts.college_conference == 'Pacific Ten Conference')] = 'Pacific Twelve Conference'


### Feature Engineering

While athletic testing is super important, not all scores are the same. A player who runs a 4.5 40 yard dash at 250 lbs is much more impressive than a player who does it at 225 lbs. So I created 3 new scores that combine both weight and an athletic test score.

In [78]:
drafts['speed'] = (drafts.weight * 200) / (drafts.forty_yd ** 4)
drafts['shuttle_agility'] = (drafts.weight * 200) / (drafts.shuttle ** 4)
drafts['cone_agility'] = (drafts.weight * 200) / (drafts.cone ** 3)

### Create Round classes (Rounds 1-3: class 2, Rounds 4-7: class 1, Undrafted: class 0)

In [79]:
drafts.Round[drafts.Round <= 3] = 2

C:\Users\capta\AppData\Local\Temp\ipykernel_21532\874012890.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.Round[drafts.Round <= 3] = 2


In [80]:
drafts.Round[drafts.Round == 8] = 0

C:\Users\capta\AppData\Local\Temp\ipykernel_21532\2272459413.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.Round[drafts.Round == 8] = 0


In [81]:
drafts.Round[drafts.Round >= 4] = 1

C:\Users\capta\AppData\Local\Temp\ipykernel_21532\254994376.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  drafts.Round[drafts.Round >= 4] = 1


### Write DataFrame to csv (for Neural Network) and parquet files

In [82]:
drafts.to_csv('./Data/draft0522.csv')

In [83]:
drafts.to_parquet('./Data/draft0522.parquet')